# local version of LFP setup (shouldn't use for longer sessions)

In [2]:
import pandas as pd; pd.set_option('display.max_columns', 30)
import numpy as np
import sys
import os
import re
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
from copy import copy
from scipy import stats,signal
from ptsa.data.timeseries import TimeSeries
import time
import mat73 # this loads .mat files as dicts
import warnings
warnings.filterwarnings("ignore", message="Unable to parse")
warnings.filterwarnings("ignore", message="Invalid")

sys.path.append('/Volumes/GoogleDrive/My Drive/Fried/python_code/modules/')
%load_ext autoreload
%autoreload
from general import *

import neuralynx_io


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## get the electrode names

In [3]:
dir_path = '/Volumes/data/NLData/D566/EXP7_Movie_24_Pre_Sleep/2023-08-05_22-00-04/'
dir_path = '/Volumes/data/NLData/D566/EXP9_Movie_24_Post_Sleep/2023-08-06_08-44-52/'
dir_path = '/Volumes/data/NLData/D566/EXP4_Morpheme_blocks 4,6,1-3/Blocks 4,6,1-3/2023-08-04_13-27-55/combinedData'

file_extension = '.ncs'
min_size_mb = 0.5  # MB
max_size_mb = 50  # MB # should not be more than 29.4MB for macros...since that's size after 2 hours when NLX creates a new file

files = find_files_in_range(dir_path, file_extension, min_size_mb, max_size_mb)

# Extract possible electrode names
matches = [re.search(r'([^/]+?)[_.]', os.path.basename(file)).group(1) for file in files] # if you want numbers with electrode names
# remove electrode numbers if they exist
for i_match,match in enumerate(matches):
    if match[-1].isdigit():
        matches[i_match] = match[:-1]
possible_electrode_names = list(set(matches))
print(possible_electrode_names)

['C', 'LSTG', 'PZ', 'RA', 'LA', 'HR', 'LOF', 'RMH', 'LFSG', 'Ez', 'LOPR', 'ROF', 'LAC', 'RAI', 'LMH']


## manually input the array of the electrode names using the last printout to get all filenames

In [4]:
electrode_prefixes = ['LAC', 'LOF', 'LOPR', 'LMH', 'RA', 'LA', 'LFSG', 'RMH', 'ROF', 'LSTG', 'RAI']

filtered_files = [filename for filename in files if any(os.path.basename(filename).startswith(prefix) for prefix in electrode_prefixes)]
filtered_files[0:20]

# create a mask that identifies the last electrode in each group so can skip that one for bipolar

def extract_info(filepath):
    basename = os.path.basename(filepath)
    match = re.search(r"([A-Za-z]+)(\d+)_?\d*\.ncs$", basename)
    if match:
        prefix, number = match.groups()
        return prefix, int(number), filepath
    else:
        raise ValueError(f"Unexpected filename format: {basename}")

file_info = [extract_info(path) for path in filtered_files]

# Identify the max number for each prefix
max_numbers = {}
for prefix, number, _ in file_info:
    if prefix not in max_numbers:
        max_numbers[prefix] = number
    else:
        max_numbers[prefix] = max(max_numbers[prefix], number)

# Generate the mask based on these maximum channel numbers
last_electrode_mask = [1 if number == max_numbers[prefix] else 0 for prefix, number, _ in file_info]
print(last_electrode_mask)

print('Data length: '+str(len(neuralynx_io.load_ncs(filtered_files[0])['data'])))
print('(which is '+str(len(neuralynx_io.load_ncs(filtered_files[0])['data'])/2000/60)+' minutes of 2000 Hz sampled data')

['/Volumes/data/NLData/D566/EXP4_Morpheme_blocks 4,6,1-3/Blocks 4,6,1-3/2023-08-04_13-27-55/combinedData/LA1.ncs',
 '/Volumes/data/NLData/D566/EXP4_Morpheme_blocks 4,6,1-3/Blocks 4,6,1-3/2023-08-04_13-27-55/combinedData/LA2.ncs',
 '/Volumes/data/NLData/D566/EXP4_Morpheme_blocks 4,6,1-3/Blocks 4,6,1-3/2023-08-04_13-27-55/combinedData/LA3.ncs',
 '/Volumes/data/NLData/D566/EXP4_Morpheme_blocks 4,6,1-3/Blocks 4,6,1-3/2023-08-04_13-27-55/combinedData/LA4.ncs',
 '/Volumes/data/NLData/D566/EXP4_Morpheme_blocks 4,6,1-3/Blocks 4,6,1-3/2023-08-04_13-27-55/combinedData/LA5.ncs',
 '/Volumes/data/NLData/D566/EXP4_Morpheme_blocks 4,6,1-3/Blocks 4,6,1-3/2023-08-04_13-27-55/combinedData/LA6.ncs',
 '/Volumes/data/NLData/D566/EXP4_Morpheme_blocks 4,6,1-3/Blocks 4,6,1-3/2023-08-04_13-27-55/combinedData/LA7.ncs',
 '/Volumes/data/NLData/D566/EXP4_Morpheme_blocks 4,6,1-3/Blocks 4,6,1-3/2023-08-04_13-27-55/combinedData/LA8.ncs',
 '/Volumes/data/NLData/D566/EXP4_Morpheme_blocks 4,6,1-3/Blocks 4,6,1-3/2023-08-

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]
Data length: 17255424
(which is 143.7952 minutes of 2000 Hz sampled data


## load macro LFP, decimate it to 1 kHz, and create a TimeSeries object

In [5]:
# from ptsa.data.timeseries import TimeSeries

bipolar_ref = 1
desired_samplerate = 1000 # Hz. 2000 Hz is default. Decimating to 1000 Hz takes ~15% longer
# time_range = range(int(1e6),int(2e6)) # range(0,len(neuralynx_io.load_ncs(filtered_files[0])['data']))
time_range = range(0,len(neuralynx_io.load_ncs(filtered_files[0])['data'])) # full data

begin_time = time.time()

current_sr = neuralynx_io.load_ncs(filtered_files[0])['sampling_rate'] # NLX micros at 32K, macros at 2k
lfp_mat = []
for i_file in range(len(filtered_files)):
    if bipolar_ref == 1:
        if last_electrode_mask[i_file] == 0:
            temp_mat = neuralynx_io.load_ncs(filtered_files[i_file])['data'][time_range] - \
                neuralynx_io.load_ncs(filtered_files[i_file+1])['data'][time_range] 
        else:
            print("skipping "+filtered_files[i_file]+" since we're doing bipolar referencing")
            continue # go on to next electrode since we have all the bipolar pairs for this one
    else:
        temp_mat = neuralynx_io.load_ncs(filtered_files[i_file])['data'][time_range]
        
    temp_mat = signal.decimate(temp_mat,int(current_sr/desired_samplerate)) 
    lfp_mat = superVstack(lfp_mat,temp_mat)
      
time_in_sec = np.linspace(1, np.shape(lfp_mat)[1], np.shape(lfp_mat)[1]) / desired_samplerate
channels = np.arange(np.shape(lfp_mat)[0])

lfp_mat = TimeSeries(lfp_mat,
                dims=('channel', 'time'),
                coords={'channel':channels,
                        'time':time_in_sec,
                        'samplerate':desired_samplerate})
print('Shape of LFP matrix (channel X time):')
print(np.shape(lfp_mat))

print('Time to run: '+str(time.time()-begin_time))

skipping /Volumes/data/NLData/D566/EXP4_Morpheme_blocks 4,6,1-3/Blocks 4,6,1-3/2023-08-04_13-27-55/combinedData/LA8.ncs since we're doing bipolar referencing
skipping /Volumes/data/NLData/D566/EXP4_Morpheme_blocks 4,6,1-3/Blocks 4,6,1-3/2023-08-04_13-27-55/combinedData/LAC9.ncs since we're doing bipolar referencing
skipping /Volumes/data/NLData/D566/EXP4_Morpheme_blocks 4,6,1-3/Blocks 4,6,1-3/2023-08-04_13-27-55/combinedData/LFSG7.ncs since we're doing bipolar referencing
skipping /Volumes/data/NLData/D566/EXP4_Morpheme_blocks 4,6,1-3/Blocks 4,6,1-3/2023-08-04_13-27-55/combinedData/LMH7.ncs since we're doing bipolar referencing
skipping /Volumes/data/NLData/D566/EXP4_Morpheme_blocks 4,6,1-3/Blocks 4,6,1-3/2023-08-04_13-27-55/combinedData/LOF8.ncs since we're doing bipolar referencing
skipping /Volumes/data/NLData/D566/EXP4_Morpheme_blocks 4,6,1-3/Blocks 4,6,1-3/2023-08-04_13-27-55/combinedData/LOPR7.ncs since we're doing bipolar referencing
skipping /Volumes/data/NLData/D566/EXP4_Morph

## filter for line noise, grab power at given frequencies, and append across frequencies

In [ ]:
from ptsa.data.filters import ButterworthFilter
from ptsa.data.filters import MorletWaveletFilter
from scipy.stats import zscore

# buf = 1000  #to remove edge effects during wavelet convolution # not doing this by events so no buffers needed
spec_freqs = np.logspace(np.log10(3), np.log10(180), 8) # spectral frequencies used in Ezzyat et al 2018
filt_range = [58., 62.] # don't worry about 120 and 180 Hz harmonics since not sampled in spec_freqs

lfp_mat = ButterworthFilter(freq_range=filt_range, filt_type='stop', order=4).filter(timeseries=lfp_mat)

#     # downsample EEG...could do this if necessary to save processing power since we're going to 
#     # bin down to 33.3 ms bins eventually anyway
#     eeg_ptsa = eeg_ptsa.resampled(resampled_rate=500.)
         
#Get spectral power

pow_wavelet = MorletWaveletFilter(freqs=spec_freqs, width=5, output='power', cpus=2).filter(timeseries=lfp_mat)
#output is freqs, events, elecs, and time
# e.g. (8, 240, 141, 700) is 8 logarithmically spaced freqs from 3:180, 240 is trials, 141 elecs, 700 samples (1400 ms each typically)

#remove buffer period added to beginning and end of each trial
#     pows = pow_wavelet.data[:, :, :, int((buf/1000.)*sr):-1*int((buf/1000.)*sr)]  
#     pows = pow_wavelet.isel(time = ((pow_wavelet.time>=start_time) & (pow_wavelet.time<=end_time)))
#     pows = np.log10(pows)

pow_wavelet = np.log10(pow_wavelet)

#     #Average power across the whole interval
avg_pows = pow_wavelet.mean('time')
#     avg_pows = np.squeeze(avg_pows)

z_pows = (pow_wavelet-avg_pows)/pow_wavelet.std('time')
# z_pows = zscore(pow_wavelet, axis='time')  #z-score across time
#     z_pows = zscore(avg_pows, axis=1)  #z-score power values across events (important to do within-session)
#     # freqs X trials X electrodes (e.g. 8 X 240 X 141)
#     z_pows = (avg_pows-avg_pows.mean('event'))/avg_pows.std('event')

print(np.shape(z_pows)) # output is frequency X channel X time
# append frequencies together to create time X feature matrix
z_pows = np.array(z_pows).swapaxes(0,2).swapaxes(1,2).reshape(np.shape(pow_wavelet)[2], -1)
print(np.shape(z_pows))


In [ ]:
elec_num = np.shape(pow_wavelet)[1]
plt.figure(figsize=(15, 4.5)); ax=plt.subplot(111)
ax1 = ax.matshow(z_pows.swapaxes(0,1), aspect='auto', cmap='RdBu_r', vmin=-3, vmax=3)
plt.ylabel(str(elec_num)+' electrodes appended 8x by frequencies')
plt.xlabel('Time (ms)')
for ln in range(8):    
    plt.plot([0, len(z_pows)],[elec_num*ln, elec_num*ln],color=(0.5,0.5,0.5)) 
plt.colorbar(ax1)
print("Seconds since started compiling = ", time.time()-begin_time)
a=1;